## Extract region of -2:+3 for experimental and reference

BBCA+1BW (where B is C/G/T, and W is A/T)  
http://genesdev.cshlp.org/content/31/13/1289.full

In [7]:
## Parameters specific to where your folders are and your data
parameter_file = '../../parameters/params.yaml'
import yaml
import sys
import os
with open(parameter_file,'r') as f:
    doc = yaml.load(f)

#p = dic2obj(**doc)

#data_folder = doc['data_folder']
#tissues = doc['tissues'].split(',')
#sys.path.append(doc['pipeline_path'])
ref_fa = doc['GENOME_FA']
#anno_gff=doc['annotation']
#mRNA_peak_file = doc["mRNA_peak_file"]
#tss_annotation = doc['tss_annotation']


import sys
import pandas as pd
import matplotlib
import seaborn as sns
import pickle
import numpy as np
import matplotlib as mpl
from matplotlib import pyplot as plt
from itertools import product
import glob
import re
from os.path import join
from matplotlib_venn import venn2
from matplotlib import rcParams
import inspect
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
sys.setrecursionlimit(3000)
%load_ext autoreload
%autoreload 2
rcParams['figure.figsize'] = 8, 6
import tqdm

from os.path import basename
##mpl.use('Agg')
#mpl.style.use('ggplot')
#mpl.style.use('fivethirtyeight')
#from Homer import *

homer_path =  "/data2/resources/software/homer/v4.11_10242019/bin/"
os.environ['PATH'] = f"{homer_path}:{os.environ['PATH']}"
!which homer

from tss.utils.wrap_findMotifs import run_findMotifs

/home/isshamie/software/anaconda2/envs/TSS/lib/python3.7/site-packages/ipykernel_launcher.py:7: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  import sys


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
/data2/resources/software/homer/v4.11_10242019/bin//homer


In [8]:
os.chdir(doc["results"])
save_dir = "v02_de_novo_motifs/alt/eTSS_all/"

#save_dir = "Results/homer_motifs/"
if not os.path.exists(save_dir):
    os.mkdir(save_dir)

## Read experimental bed file and create its companion refseq region

In [9]:
#mrna_bed_f = "../Results/Figures/Figure3/A.TSS1_mrna"
exp_bed_f = "alt/eTSS/TSS.exp.bed"

## Create RefSeq peak file for this
meta_f = "alt/eTSS/TSS.meta.tsv"

mrna_bed_f = "alt/eTSS/refseq_centered.TSS.bed"

#mrna_filt = "Results/Figures/Figure3/A.mrna"
#create_output.exp_bed_to_refseq(exp_bed_f,meta_f,refseq_f=mRNA_peak_file,save_f=mrna_bed_f,is_unique=True)

### Create narrow bed file for hg38

In [10]:
hg38_f = "/data/isshamie/genome/hg38/mRNA_final.peak"
hg38 = pd.read_csv("/data/isshamie/genome/hg38/mRNA_final.peak", sep="\t")
hg38["Stat"] = 0
hg38 = hg38[["Chr", "Start", "End","Name", "Stat", "Strand"]]
hg38 = hg38.drop_duplicates(subset=("Chr","Start", "End", "Strand"))
hg38 = hg38.rename({"Name":"ID"}, axis=1)

#hg38_bed_f = os.path.join(save_dir, "hg38.bed")
#write_bed_file(hg38, hg38_bed_f, use_index=False)


## Run findMotifs in Homer

In [5]:
# curr_dir = os.path.join(save_dir, "eTSS_bg_rTSS_nocpg")
# run_findMotifs(exp_bed_f, out_dir=curr_dir,ref_fa=ref_fa, bg=mrna_bed_f, size=200,num_cores=12)


exp_dir = os.path.join(save_dir, "eTSS_motifs_nocpg")
run_findMotifs(exp_bed_f, out_dir=exp_dir, mset=True, size=(-300,100), ref_fa=ref_fa,num_cores=12)

ref_dir = os.path.join(save_dir, "rTSS_motifs_nocpg")
run_findMotifs(mrna_bed_f, out_dir=ref_dir, mset=True, size=(-300,100), ref_fa=ref_fa,num_cores=12)

hg38_out = os.path.join(save_dir, "hg38_nocpg")
hg38_genome = "/data/isshamie/genome/hg38/GCF_000001405.38_GRCh38.p12_genomic.fna"
run_findMotifs(hg38_f, out_dir=hg38_out, mset=True, size=(-300,100), ref_fa=hg38_genome,num_cores=12, 
              mset=True, len_mo='large')

#wrap_findMotifs.run_findMotifs(tis_bed, tis_out, ref_fa, args=None,  bg=out_bed, mset=True, size=(-300,100), num_cores=16)
# wrap_findMotifs.run_findMotifs(out_merge+'.merge', join(prom_enr_dir,t, 'homer_out'), ref_fa, args=None,  bg=None, 
#                                size=(-300,100), mset=True, len_mo='large', num_cores=12)

nohup findMotifsGenome.pl alt/eTSS/TSS.exp.bed /data/isshamie/genome/ncbi_anno_103/GCF_003668045.1_CriGri-PICR_genomic.fna v02_de_novo_motifs/alt/eTSS_all/eTSS_motifs_nocpg -size -300,100 -len 6,8,10 -p 12  -mset vertebrate > v02_de_novo_motifs/alt/eTSS_all/eTSS_motifs_nocpg.log
nohup findMotifsGenome.pl alt/eTSS/refseq_centered.TSS.bed /data/isshamie/genome/ncbi_anno_103/GCF_003668045.1_CriGri-PICR_genomic.fna v02_de_novo_motifs/alt/eTSS_all/rTSS_motifs_nocpg -size -300,100 -len 6,8,10 -p 12  -mset vertebrate > v02_de_novo_motifs/alt/eTSS_all/rTSS_motifs_nocpg.log
nohup findMotifsGenome.pl /data/isshamie/genome/hg38/mRNA_final.peak /data/isshamie/genome/hg38/GCF_000001405.38_GRCh38.p12_genomic.fna v02_de_novo_motifs/alt/eTSS_all/hg38_nocpg -size -300,100 -len 6,8,10 -p 12  -mset vertebrate > v02_de_novo_motifs/alt/eTSS_all/hg38_nocpg.log


In [6]:
exp_dir = os.path.join(save_dir, "eTSS_motifs")
run_findMotifs(exp_bed_f, out_dir=exp_dir,mset=True, size=(-300,100),ref_fa=ref_fa,num_cores=12, args=["-cpg"])

ref_dir = os.path.join(save_dir, "rTSS_motifs")
run_findMotifs(mrna_bed_f, out_dir=ref_dir,mset=True, size=(-300,100),ref_fa=ref_fa,num_cores=12, args=["-cpg"])

curr_dir = os.path.join(save_dir, "eTSS_bg_rTSS")
run_findMotifs(exp_bed_f, out_dir=curr_dir,mset=True, size=(-300,100),ref_fa=ref_fa, bg=mrna_bed_f,num_cores=12, args=["-cpg"])



nohup findMotifsGenome.pl alt/eTSS/TSS.exp.bed /data/isshamie/genome/ncbi_anno_103/GCF_003668045.1_CriGri-PICR_genomic.fna v02_de_novo_motifs/alt/eTSS_all/eTSS_motifs -size -300,100 -len 6,8,10 -p 12  -mset vertebrate-cpg  > v02_de_novo_motifs/alt/eTSS_all/eTSS_motifs.log
nohup findMotifsGenome.pl alt/eTSS/refseq_centered.TSS.bed /data/isshamie/genome/ncbi_anno_103/GCF_003668045.1_CriGri-PICR_genomic.fna v02_de_novo_motifs/alt/eTSS_all/rTSS_motifs -size -300,100 -len 6,8,10 -p 12  -mset vertebrate-cpg  > v02_de_novo_motifs/alt/eTSS_all/rTSS_motifs.log
nohup findMotifsGenome.pl alt/eTSS/TSS.exp.bed /data/isshamie/genome/ncbi_anno_103/GCF_003668045.1_CriGri-PICR_genomic.fna v02_de_novo_motifs/alt/eTSS_all/eTSS_bg_rTSS -size -300,100 -len 6,8,10 -p 12  -mset vertebrate -bg alt/eTSS/refseq_centered.TSS.bed -cpg  > v02_de_novo_motifs/alt/eTSS_all/eTSS_bg_rTSS.log


## Get the specific matches of the top motifs
Done using: `annotatePeaks.pl ERalpha.peaks hg18 -m motif1.motif > outputfile.txt`


In [7]:
# curr_motif = ""
# output_f = join(save_dir, "eTSS_motifs_nocpg", curr_motif+'.txt')
# cmd = f"annotatePeaks.pl {exp_bed_f} {ref_fa} -m {curr_motif} > {output_f}"